# Day 2 - Document Q&A with RAG using Chroma
歡迎回來參加 Kaggle 5 天生成式 AI 課程！

**注意**：Day 1 筆記本中包含了許多設定 Kaggle 筆記本的相關資訊。如果你遇到任何問題，請參考[那邊的故障排除步驟](https://www.kaggle.com/code/markishere/day-1-prompting#Get-started-with-Kaggle-notebooks)。

大型語言模型（LLM）有兩個主要的限制：1) 它們僅「知道」訓練期間接觸過的資訊，2) 輸入內容的長度有限。解決這兩個限制的方法之一，就是採用檢索增強生成技術（Retrieval Augmented Generation, RAG）。一個 RAG 系統包含三個階段：

1. 索引建立
2. 文件檢索
3. 回答生成

索引建立是在查詢前預先進行的，可以讓你在查詢時迅速查找相關資訊。當收到查詢後，系統會檢索出相關文件，將文件內容與指示與使用者查詢結合，再利用大型語言模型生成符合查詢需求的自然語言答案。這使得你可以提供模型訓練期間未曾見過的資訊，例如產品專屬知識或即時天氣更新。

在這個筆記本中，你將使用 Gemini API 建立向量資料庫，從資料庫中檢索文件來回答問題，並生成最終答案。你會使用 [Chroma](https://docs.trychroma.com/)，這是一個開源的向量資料庫。利用 Chroma，你可以儲存嵌入向量以及元資料，對文件和查詢進行嵌入，並搜尋文件。

## 幫助

**常見問題請參考** [FAQ 與故障排除指南](https://www.kaggle.com/code/markishere/day-0-troubleshooting-and-faqs)。

## 環境設定

首先，請安裝 ChromaDB 與 Gemini API 的 Python SDK。

In [1]:
!pip uninstall -qqy jupyterlab kfp  # 移除未使用且可能衝突的套件
!pip install -qU "google-genai==1.7.0" "chromadb==0.6.3"

In [7]:
from google import genai
from google.genai import types

from IPython.display import Markdown

genai.__version__

'1.7.0'

### 設定 API 金鑰

請先將 `YOUR_GOOGLE_API_KEY` 替換成自己的 API 金鑰。  
若你還沒有 API 金鑰，可前往 [AI Studio](https://aistudio.google.com/app/apikey) 取得，詳情請參考 [Gemini API 文件](https://ai.google.dev/gemini-api/docs/api-key)。


In [1]:
GOOGLE_API_KEY = 'YOUR_GOOGLE_API_KEY'

### 探索可用模型

在本範例中，你會使用 [`embedContent`](https://ai.google.dev/api/embeddings#method:-models.embedcontent) API 方法來計算嵌入向量。請透過 [`models.list`](https://ai.google.dev/api/models#method:-models.list) 端點尋找支援此方法的模型。你也可以在 [模型頁面](https://ai.google.dev/gemini-api/docs/models/gemini#text-embedding) 中找到更多關於嵌入模型的資訊。

`text-embedding-004` 是目前最新且普遍可用的嵌入模型，因此在這個範例中你將使用它，但也可以試試實驗性的 `gemini-embedding-exp-03-07` 模型。

> Google Gemini 的 text-embedding-004 目前不建議用繁體中文


In [8]:
client = genai.Client(api_key=GOOGLE_API_KEY)

for m in client.models.list():
    if "embedContent" in m.supported_actions:
        print(m.name)

models/embedding-001
models/text-embedding-004
models/gemini-embedding-exp-03-07
models/gemini-embedding-exp


### 資料
以下是一組小型文件資料集，你將用它們來建立嵌入資料庫。


In [18]:
DOCUMENT1 = "Operating the Climate Control System  Your Googlecar has a climate control system that allows you to adjust the temperature and airflow in the car. To operate the climate control system, use the buttons and knobs located on the center console.  Temperature: The temperature knob controls the temperature inside the car. Turn the knob clockwise to increase the temperature or counterclockwise to decrease the temperature. Airflow: The airflow knob controls the amount of airflow inside the car. Turn the knob clockwise to increase the airflow or counterclockwise to decrease the airflow. Fan speed: The fan speed knob controls the speed of the fan. Turn the knob clockwise to increase the fan speed or counterclockwise to decrease the fan speed. Mode: The mode button allows you to select the desired mode. The available modes are: Auto: The car will automatically adjust the temperature and airflow to maintain a comfortable level. Cool: The car will blow cool air into the car. Heat: The car will blow warm air into the car. Defrost: The car will blow warm air onto the windshield to defrost it."
DOCUMENT2 = 'Your Googlecar has a large touchscreen display that provides access to a variety of features, including navigation, entertainment, and climate control. To use the touchscreen display, simply touch the desired icon.  For example, you can touch the "Navigation" icon to get directions to your destination or touch the "Music" icon to play your favorite songs.'
DOCUMENT3 = "Shifting Gears Your Googlecar has an automatic transmission. To shift gears, simply move the shift lever to the desired position.  Park: This position is used when you are parked. The wheels are locked and the car cannot move. Reverse: This position is used to back up. Neutral: This position is used when you are stopped at a light or in traffic. The car is not in gear and will not move unless you press the gas pedal. Drive: This position is used to drive forward. Low: This position is used for driving in snow or other slippery conditions."

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]

## 使用 ChromaDB 建立嵌入資料庫

首先，建立一個 [自訂函式](https://docs.trychroma.com/guides/embeddings#custom-embedding-functions) 用以透過 Gemini API 生成嵌入向量。在這個任務中，你要實作一個檢索系統，因此針對*文件*生成嵌入時，其 `task_type` 設為 `retrieval_document`；稍後針對*查詢*生成嵌入時，則使用 `retrieval_query`。請參考 [API 文件](https://ai.google.dev/api/embeddings#v1beta.TaskType) 以獲得完整支援的任務類型清單。

關鍵說明：資料庫中的項目稱為「文件」，這些文件會先被寫入資料庫，稍後再檢索。查詢則是文字搜尋字串，可以是簡單關鍵字或描述所需文件的文字內容。


In [10]:
from chromadb import Documents, EmbeddingFunction, Embeddings
from google.api_core import retry

from google.genai import types

# 定義一個輔助函式，當每分鐘額度不足時進行重試
is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

class GeminiEmbeddingFunction(EmbeddingFunction):
    # 指定是否為文件生成嵌入，或為查詢生成嵌入
    document_mode = True

    @retry.Retry(predicate=is_retriable)
    def __call__(self, input: Documents) -> Embeddings:
        if self.document_mode:
            embedding_task = "retrieval_document"
        else:
            embedding_task = "retrieval_query"

        response = client.models.embed_content(
            model="models/text-embedding-004",
            contents=input,
            config=types.EmbedContentConfig(
                task_type=embedding_task,
            ),
        )
        return [e.values for e in response.embeddings]

現在建立一個使用 GeminiEmbeddingFunction 的 [Chroma 資料庫](https://docs.trychroma.com/getting-started) client，並把上面定義好的文件加進資料庫裡。

In [11]:
import chromadb

DB_NAME = "googlecardb"

embed_fn = GeminiEmbeddingFunction()
embed_fn.document_mode = True

chroma_client = chromadb.Client()
db = chroma_client.get_or_create_collection(name=DB_NAME, embedding_function=embed_fn)

db.add(documents=documents, ids=[str(i) for i in range(len(documents))])


你可以透過檢查資料庫的內容，確認文件是否已經成功寫入。以下檢查寫入的筆數。

In [12]:
print("資料庫中文件數：", db.count())
# 你也可以預覽資料內容：
# db.peek(0)

3

## 檢索(Retrieval)：尋找相關文件

要在 Chroma 資料庫中進行搜尋，請呼叫 `query` 方法。注意，此時要切換至 `retrieval_query` 模式來生成查詢的嵌入向量。


In [17]:
# 切換至查詢模式以生成嵌入向量
embed_fn.document_mode = False

# 使用指定查詢搜尋 Chroma 資料庫
query = "How do you use the touchscreen to play music?"

result = db.query(query_texts=[query], n_results=1)
[all_passages] = result["documents"]

Markdown(all_passages[0])

Your Googlecar has a large touchscreen display that provides access to a variety of features, including navigation, entertainment, and climate control. To use the touchscreen display, simply touch the desired icon.  For example, you can touch the "Navigation" icon to get directions to your destination or touch the "Music" icon to play your favorite songs.

## 增強生成(Augmented generation)：回答問題

既然你已經從文件集中檢索到一個相關段落（檢索步驟），接下來就可以組合生成提示，讓 Gemini API 根據該提示生成最終答案。請注意，在此例中僅檢索到單一段落。在實務上，尤其當底層資料集規模龐大時，你會希望檢索多個結果，並讓 Gemini 模型判斷哪些段落與問題回答最相關。因此，即使部分檢索結果與問題無直接關係也沒關係，此生成步驟會忽略不相干內容。


In [14]:
query_oneline = query.replace("\n", " ")

# 此提示可用來指示模型的語氣、應遵循或避免的主題等。
prompt = f"""你是一個友善且資訊豐富的機器人，請利用以下參考段落回答問題。
請務必以完整句子回覆，內容要詳盡且包含所有相關背景資訊。
但由於對象是非技術使用者，請分解複雜概念，並採用友善且口語化的語氣。如果段落與答案無關，你可以忽略它。

問題：{query_oneline}
"""

# 將檢索到的文件段落加入提示中
for passage in all_passages:
    passage_oneline = passage.replace("\n", " ")
    prompt += f"段落：{passage_oneline}\n"

print(prompt)

你是一個友善且資訊豐富的機器人，請利用以下參考段落回答問題。
請務必以完整句子回覆，內容要詳盡且包含所有相關背景資訊。
但由於對象是非技術使用者，請分解複雜概念，並採用友善且口語化的語氣。如果段落與答案無關，你可以忽略它。

問題：How do you use the touchscreen to play music?
段落：Your Googlecar has a large touchscreen display that provides access to a variety of features, including navigation, entertainment, and climate control. To use the touchscreen display, simply touch the desired icon.  For example, you can touch the "Navigation" icon to get directions to your destination or touch the "Music" icon to play your favorite songs.



接著使用 generate_content 方法來產生這個問題的回答。

In [15]:
answer = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=prompt)

Markdown(answer.text)

要用 Googlecar 的觸控螢幕來播放音樂非常簡單喔！想像一下，你的 Googlecar 有一個很大的平板電腦螢幕，那就是觸控螢幕啦！這個螢幕有很多功能，像是幫你導航、讓你聽音樂、調整冷氣等等。

如果你想聽音樂，只要用手指輕輕點一下螢幕上的「Music」（音樂）圖示就可以了。點下去之後，你就可以選擇你想聽的歌曲，盡情享受音樂囉！就像平常用手機或平板電腦聽音樂一樣簡單！


## 下一步

恭喜你成功建立一個檢索增強生成應用程式！

若想進一步了解如何在 Gemini API 中使用嵌入技術，請參閱 [嵌入入門](https://ai.google.dev/gemini-api/docs/embeddings)；或若想學習更多基礎知識，請參考機器學習速成課程中的 [嵌入章節](https://developers.google.com/machine-learning/crash-course/embeddings)。

另外，若你想建立託管式的 RAG 系統，請參閱 Gemini API 中的 [語意檢索服務](https://ai.google.dev/gemini-api/docs/semantic_retrieval)。你可以透過單次請求在你自己的文件上實作問答，或是託管一個資料庫以獲得更快速的回應。

*- [Mark McD](https://linktr.ee/markmcd)*

In [ ]:
# Copyright 2025 Google LLC.

# @title 根據 Apache License, Version 2.0（以下簡稱「授權條款」）取得授權；
# 除非遵循授權條款，否則不得使用此檔案。
# 你可以從以下網址取得授權條款副本：
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# 除非適用法律要求或書面同意，本軟體皆按「現狀」提供，
# 不附帶任何形式的明示或暗示保證。
# 請參閱授權條款以了解具體權限與限制。